# Code for making drift data
## Make bright images and camera dust images from wire raw images 

# ------------------------------------------------------------------------------------------

## Preparing

In [ ]:
import cv2
import os
import numpy as np
from PIL import Image
import glob

In [ ]:
def adjust_gamma(image, gamma=1.0):
    """gamma adjustment（adjust overall brightness）"""
    invGamma = 1.0 / gamma
    table = np.array([(i / 255.0) ** invGamma * 255 for i in np.arange(256)]).astype("uint8")
    return cv2.LUT(image, table)

def adjust_brightness_hsv(image, factor=1.0):
    """change V in HSV space"""
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV).astype(np.float32)
    hsv[..., 2] *= factor
    hsv[..., 2] = np.clip(hsv[..., 2], 0, 255)
    return cv2.cvtColor(hsv.astype(np.uint8), cv2.COLOR_HSV2BGR)

def change_color_temperature(image, blue_scale=1.0, red_scale=1.0):
    """change color temperature (balance of blue and red)"""
    b, g, r = cv2.split(image.astype(np.float32))
    b *= blue_scale
    r *= red_scale
    b = np.clip(b, 0, 255)
    r = np.clip(r, 0, 255)
    return cv2.merge([b.astype(np.uint8), g.astype(np.uint8), r.astype(np.uint8)])

def apply_spotlight(image, center=None, strength=0.5, radius=200):
    """make partial spotlight"""
    h, w = image.shape[:2]
    if center is None:
        center = (w // 2, h // 2)
    mask = np.zeros((h, w), dtype=np.float32)
    cv2.circle(mask, center, radius, (1.0,), thickness=-1, lineType=cv2.LINE_AA)
    mask = cv2.GaussianBlur(mask, (0, 0), radius / 2)
    mask = (1 - strength) + strength * mask
    result = image.astype(np.float32) * mask[..., np.newaxis]
    return np.clip(result, 0, 255).astype(np.uint8)

def simulate_light_change(image, gamma=0.9, hsv_factor=1.1, blue_scale=0.95, red_scale=1.05, 
                          spotlight_center=None, spotlight_strength=0.3, spotlight_radius=150):
    """reproduct brightness shift by using all functions above"""
    # 1. gamma adjust
    img_gamma = adjust_gamma(image, gamma)
    
    # 2. adjust brightness in HSV space
    img_hsv = adjust_brightness_hsv(img_gamma, hsv_factor)
    
    # 3. change color temperature
    img_temp = change_color_temperature(img_hsv, blue_scale, red_scale)
    
    # 4. spotlight effect
    img_final = apply_spotlight(img_temp, center=spotlight_center, 
                                 strength=spotlight_strength, radius=spotlight_radius)
    
    return img_final

# ------------------------------------------------------------------------------------------

## Making bright images

In [ ]:
# folder
src_folder = r"C:\Users\pass" #folder containing wire raw images
bright_folder = r"C:\Users\pass" # output folder 

In [ ]:
#experiment
experimental_fname = os.listdir(src_folder)[0]
experimental_path = os.path.join(src_folder, experimental_fname)
experimental_image = cv2.imread(experimental_path)
processed_experimental_image = simulate_light_change(experimental_image, gamma=1.4, hsv_factor=1.6, blue_scale=1, red_scale=1, 
                          spotlight_center=(50, experimental_image.shape[0]-50), spotlight_strength=0.2, spotlight_radius=300)

plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.imshow(experimental_image)
plt.subplot(1, 2, 2)
plt.imshow(processed_experimental_image)

In [ ]:
#make bright image
for fname in os.listdir(src_folder):
    if not fname.lower().endswith(".jpg"):
        continue  # skip other than jpg

    path = os.path.join(src_folder, fname)
    img = cv2.imread(path)
    if img is None:
        print(f"fail: {fname}")
        continue

    bright_img = simulate_light_change(img, gamma=1.4, hsv_factor=1.6, blue_scale=1, red_scale=1, 
                          spotlight_center=(50, experimental_image.shape[0]-50), spotlight_strength=0.2, spotlight_radius=300)
    bright_path = os.path.join(bright_folder, fname)
    cv2.imwrite(bright_path, bright_img)

print("process done!!")

# ------------------------------------------------------------------------------------------

## Making camera dust images

In [ ]:
alpha = 0.15 # parameter of alpha blending
mask_path = r"C:\Users\pass"  #import mask image

# input folder containing wire raw images
input_dirs = [
    r"C:\Users\pass",
    r"C:\Users\pass",
    r"C:\Users\pass",
]

# output folder
output_dirs = [
    r"C:\Users\pass"
    r"C:\Users\pass",
    r"C:\Users\pass",
]

# ==========================
# read mask image
# ==========================
mask_orig = Image.open(mask_path).convert("L")

# ==========================
# process each folder and make camera dust images
# ==========================
for in_dir, out_dir in zip(input_dirs, output_dirs):

    image_paths = []
    for ext in ("*.jpg", "*.jpeg", "*.png", "*.tif", "*.tiff", "*.bmp"):
        image_paths.extend(glob.glob(os.path.join(in_dir, ext)))

    print(f"\n=== {in_dir} を処理: {len(image_paths)} 枚 ===")

    for path in image_paths:

        # read input images (grey scale)
        img = Image.open(path).convert("L")

        # resize mask image if the size is different
        if img.size != mask_orig.size:
            mask_img = mask_orig.resize(img.size, Image.BILINEAR)
        else:
            mask_img = mask_orig

        # NumPy
        arr_img = np.asarray(img, dtype=np.float32)
        arr_mask = np.asarray(mask_img, dtype=np.float32)

        # alpha blending
        blended = (1 - alpha) * arr_img + alpha * arr_mask
        blended = np.clip(blended, 0, 255).astype(np.uint8)

        # Back to Pillow image
        out_img = Image.fromarray(blended, mode="L")

        # make the name of save file
        filename = os.path.basename(path)
        name, ext = os.path.splitext(filename)
        out_path = os.path.join(out_dir, f"{name}_alpha007{ext}")

        # save
        out_img.save(out_path)
        print(f"saved: {out_path}")
